# Salary prediction model for programmers
## Structure of the code
1. **Data exploration**
2. Fit regression full model (with all available columns)
3. Fit regression partial model (after grouping)<br>
4. Interesting observations
    

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv("survey_results_public.csv",low_memory=False)
data.shape

(98855, 129)

In [16]:
# Redundant columns
DROP_COLUMNS = ['CurrencySymbol','Salary', 'SalaryType', 'Respondent', 'Currency']

data.drop(DROP_COLUMNS, axis=1, inplace=True)

MULTIPLE_CHOICE = [
    'DevType','IDE', 'FrameworkWorkedWith',
    'CommunicationTools','EducationTypes','SelfTaughtTypes',
    'DatabaseWorkedWith','PlatformWorkedWith',
    'Methodology','VersionControl',
    'ErgonomicDevices','Gender',
    'SexualOrientation','RaceEthnicity', 
    'LanguageWorkedWith'
]


# These columns are not useful / hard to interpretate.
DROP_COLUMNS = [
'AssessJob1','AssessJob2','AssessJob3', 'AssessJob4', 'AssessJob5', 'AssessJob6', 'AssessJob7', 'AssessJob8', 'AssessJob9', 'AssessJob10',
'AssessBenefits1','AssessBenefits2', 'AssessBenefits3', 'AssessBenefits4', 'AssessBenefits5', 'AssessBenefits6', 'AssessBenefits7',
'AssessBenefits8', 'AssessBenefits9', 'AssessBenefits10', 'AssessBenefits11',
'JobContactPriorities1', 'JobContactPriorities2', 'JobContactPriorities3', 'JobContactPriorities4', 'JobContactPriorities5',
'JobEmailPriorities1', 'JobEmailPriorities2', 'JobEmailPriorities3', 'JobEmailPriorities4','JobEmailPriorities5', 'JobEmailPriorities6',
'JobEmailPriorities7',
'AdsPriorities1', 'AdsPriorities2', 'AdsPriorities3', 'AdsPriorities4', 'AdsPriorities5', 'AdsPriorities6', 'AdsPriorities7',
'AIDangerous','AIInteresting','AIResponsible','AIFuture',
'EthicsChoice','EthicsReport','EthicsResponsible','EthicalImplications',
'FrameworkDesireNextYear','LanguageDesireNextYear','DatabaseDesireNextYear','PlatformDesireNextYear',
'SurveyTooLong','SurveyEasy',
'HypotheticalTools1','HypotheticalTools2','HypotheticalTools3','HypotheticalTools4','HypotheticalTools5',
'AdsAgreeDisagree1', 'AdsAgreeDisagree2', 'AdsAgreeDisagree3',
'StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount','StackOverflowParticipate','StackOverflowJobs','StackOverflowDevStory',
'StackOverflowJobsRecommend','StackOverflowConsiderMember',
'AdsActions','AdBlockerReasons','AgreeDisagree1','AgreeDisagree2','AgreeDisagree3','JobSatisfaction','CareerSatisfaction','HopeFiveYears',
'UpdateCV','HackathonReasons','AdBlocker','AdBlockerDisable','AdBlockerReasons','AdsActions'
]

data.drop(DROP_COLUMNS, axis=1, inplace=True)
df=data[data['ConvertedSalary'].notnull()][:]


In [17]:
#divide by country type
developed_country = ['United States','Japan','Turkey','Germany','United Kingdom','France','Italy','South Korea','Spain','Canada','Australia','Netherlands','Belgium','Greece','Czech Republic','Portugal','Sweden','Austria','Switzerland','Israel','Singapore','Denmark','Finland','Norway','Ireland','New Zealand','Slovenia','Estonia','Cyprus','Luxembourg','Iceland']
developed_df = df[df['Country'].isin(developed_country)]
developing_df = df[~df['Country'].isin(developed_country)]
developed_df.drop(['Country'],axis = 1, inplace = True)
developing_df.drop(['Country'],axis = 1, inplace = True)
print(developed_df.shape)
print(developing_df.shape)


(33431, 41)
(14271, 41)


In [18]:
def NA(df):
    df.dropna(axis=1, how='all', inplace=True)
    dummy_columns = [c for c in df.columns if len(df[c].unique()) == 2]
    non_dummy = [c for c in df.columns if c not in dummy_columns]
    df[dummy_columns] = df[dummy_columns].fillna(0)
    df[non_dummy] = df[non_dummy].fillna(df[non_dummy].median())
    print(f">> Filled NaNs in {len(dummy_columns)} OHE columns with 0")
    print(f">> Filled NaNs in {len(non_dummy)} non-OHE columns with median values")

NA(developed_df)
NA(developing_df)

def coef(df):
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find index of feature columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

    # Drop those columns
    print(f">> Dropping the following columns due to high correlations: {to_drop}")
    df = df.drop(to_drop, axis=1)
    df = df[df['ConvertedSalary'] != 0]

coef(developed_df)
coef(developing_df)


>> Filled NaNs in 2 OHE columns with 0
>> Filled NaNs in 39 non-OHE columns with median values
>> Filled NaNs in 2 OHE columns with 0
>> Filled NaNs in 38 non-OHE columns with median values
>> Dropping the following columns due to high correlations: []
>> Dropping the following columns due to high correlations: []


In [20]:
print(developed_df.shape)
print(developing_df.shape)

(33431, 41)
(14271, 40)


In [19]:
developed_df.to_csv(r'/Users/jolinchen/machine_learning/project/developed_df_full.csv')
developing_df.to_csv(r'/Users/jolinchen/machine_learning/project/developing_df_full.csv')

In [11]:
%%time
# remove outliers
def remove_outliers(df):
    Q1 = df['ConvertedSalary'].quantile(0.25)
    Q3 = df['ConvertedSalary'].quantile(0.75)
    IQR = Q3 - Q1
    drop_outliers = []
    for i in df['ConvertedSalary'].index:
        if df['ConvertedSalary'][i] < Q1 - 1.5*IQR or df['ConvertedSalary'][i] > Q3 + 1.5*IQR:
            drop_outliers.append(i)
    df.drop(drop_outliers, inplace = True)

remove_outliers(developed_df)
remove_outliers(developing_df)



# Go through all object columns
def MC(df):
    for c in MULTIPLE_CHOICE:

        # Check if there are multiple entries in this column
        temp = df[c].str.split(';', expand=True)

        # Get all the possible values in this column
        new_columns = pd.unique(temp.values.ravel())
        for new_c in new_columns:
            if new_c and new_c is not np.nan:

                # Create new column for each unique column
                idx = df[c].str.contains(new_c, regex=False).fillna(False)
                df.loc[idx, f"{c}_{new_c}"] = 1

        # Info to the user
        print(f">> Multiple entries in {c}. Added {len(new_columns)} one-hot-encoding columns")

        # Drop the original column
        df.drop(c, axis=1, inplace=True)

MC(developed_df)
MC(developing_df)


>> Multiple entries in DevType. Added 22 one-hot-encoding columns
>> Multiple entries in IDE. Added 24 one-hot-encoding columns
>> Multiple entries in FrameworkWorkedWith. Added 14 one-hot-encoding columns
>> Multiple entries in CommunicationTools. Added 13 one-hot-encoding columns
>> Multiple entries in EducationTypes. Added 11 one-hot-encoding columns
>> Multiple entries in SelfTaughtTypes. Added 11 one-hot-encoding columns
>> Multiple entries in DatabaseWorkedWith. Added 23 one-hot-encoding columns
>> Multiple entries in PlatformWorkedWith. Added 28 one-hot-encoding columns
>> Multiple entries in Methodology. Added 12 one-hot-encoding columns
>> Multiple entries in VersionControl. Added 9 one-hot-encoding columns
>> Multiple entries in ErgonomicDevices. Added 6 one-hot-encoding columns
>> Multiple entries in Gender. Added 6 one-hot-encoding columns
>> Multiple entries in SexualOrientation. Added 6 one-hot-encoding columns
>> Multiple entries in RaceEthnicity. Added 9 one-hot-encodin

### Fill in NA and drop high correlation features

In [12]:
def NA(df):
    df.dropna(axis=1, how='all', inplace=True)
    dummy_columns = [c for c in df.columns if len(df[c].unique()) == 2]
    non_dummy = [c for c in df.columns if c not in dummy_columns]
    df[dummy_columns] = df[dummy_columns].fillna(0)
    df[non_dummy] = df[non_dummy].fillna(df[non_dummy].median())
    print(f">> Filled NaNs in {len(dummy_columns)} OHE columns with 0")
    print(f">> Filled NaNs in {len(non_dummy)} non-OHE columns with median values")

NA(developed_df)
NA(developing_df)

def coef(df):
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find index of feature columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

    # Drop those columns
    print(f">> Dropping the following columns due to high correlations: {to_drop}")
    df = df.drop(to_drop, axis=1)
    df = df[df['ConvertedSalary'] != 0]

coef(developed_df)
coef(developing_df)


>> Filled NaNs in 206 OHE columns with 0
>> Filled NaNs in 24 non-OHE columns with median values
>> Filled NaNs in 206 OHE columns with 0
>> Filled NaNs in 23 non-OHE columns with median values
>> Dropping the following columns due to high correlations: []
>> Dropping the following columns due to high correlations: []


In [13]:
print(developed_df.shape)
print(developing_df.shape)

(31226, 230)
(13324, 229)


In [6]:
developed_df.to_csv(r'/Users/jolinchen/machine_learning/project/developed_df_full.csv')
developing_df.to_csv(r'/Users/jolinchen/machine_learning/project/developing_df_full.csv')

In [7]:
developed_df = pd.get_dummies(developed_df)
developing_df = pd.get_dummies(developing_df)
developed_df.to_csv(r'/Users/jolinchen/machine_learning/project/developed_df.csv')
developing_df.to_csv(r'/Users/jolinchen/machine_learning/project/developing_df.csv')